# Differential Equation Machine

### batching 

* allows for calculations to be done in parallel
* distibuted caluclations
* scaleable
* gives more flexability
* Batch training allows for more Efficient training
* 


### common issues

* dead network problem where an initial bias is set to zero and is propegated through the network. the model doesnt learn

# Abstractions

* neuron
* layer
* activation function
* pack propoagation
* gradient decent
* embedder - *ie onehot embedding for normalizing and scaling
* 


### Optimizers
- _Gradient_ : First-order derivative for a multivariate objective function.
- _Step Size (alpha)_ : Hyperparameter that controls how far to move in the search space against the gradient each iteration of the algorithm.
- algorithms that organize how nerons get adjusted to training data
- function that iterates though nerons and changes weights, biases, and activation fucntions to fit data
- Stochastic Gradient Decent
- Adaptive Movement Estimation (Adam)

### Activation Functions Layer

* ReLu
    - efficient
    - popular
* Step - Hevicide Function
    - systen if functions that creates segments with steps (if theta-x > theta-z)
* Sigmoid
    - slower than relu
    - vanishing gradients
* Softmax
    - used for classification

* every neuron in the network has an activation function
* connects two nerons

* why use it?
    - allows for model to segment its self and have each neruon self determine a portion of the data
    - with only linear functions, the network can only learn linear data
    - using non linear functions allows for fitting of more complex data



### Loss

* Categorical Cross Entropy
 - takes an output from the model, does softmax, compares the normalized output to groundtruth, then finally returns how wrong the values are.


### Normalization

* y= e^x
* helps encode data with more quality.
* OneHotEncoding - encodes data into categorical numbers so that linear neral nets can train on classes.
* "Bias Trick": adding an extra dimension to the input data (usallly a 1) to implify bias.

### BackPropagation

* the process of taking the gradient of the loss with respect to weights, and the gradient of the loss with respect to inputs.

* gradient derived from the composite function of all the layers.



"Backpropagation is really one instance of a more general technique called "reverse mode differentiation" to compute derivatives of functions represented in some kind of directed graph form." - 3BlueOneBrown (Deep Learning Chapter 4)

"Essentially, backpropagation evaluates the expression for the derivative of the cost function as a product of derivatives between each layer from left to right – "backwards" – with the gradient of the weights between each layer being a simple modification of the partial products (the "backwards propagated error)."

In [ ]:
import numpy as np
import nnfs
import math


E = math.e

# sets default configuration for numpy to avoid data type errors
nnfs.init() 

np.random.seed(42)

class DenseLayer:
    def __init__(self, n_inputs, n_neurons):
        self.weights = self.weights = np.random.randn(n_inputs, n_neurons) * np.sqrt(2 / n_inputs)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases
    # takes the derivaties of the previous layer
    
    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

    def __repr__(self):
        # construct and return a string that represents the network
        # architecture
        return "NeuralNetwork: {}".format("-".join(str(l) for l in self.weights))

class ReLu:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
    
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        
        #zero the gradient where inputs are negative
        self.dinputs[self.inputs <= 0] = 0

class Sigmoid:
    def forward(self, inputs):
        self.output = 1/(1 + np.exp(-inputs))

class Softmax:
    #safe softmax prevents memory overflow
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

class Loss:
    def calculate(self, y, output):
        sample_losses = self.forward(y, output)
        data_loss = np.mean(sample_losses)
        return data_loss

class CategoricalCrossEntropyLoss(Loss):
    def forward(self, y_truth,  y_probs):
        samples = len(y_probs)
        y_pred_clipped = np.clip(y_probs, 1e-7, 1-1e-7)
        
        if len(y_truth.shape) == 1: #probabilites for target values, only if categorical labels
            correct_confidences = y_pred_clipped[range(samples), y_truth]
            
        if len(y_truth.shape) == 2: # Masking for one hot encoding
            correct_confidences = np.sum(y_pred_clipped*y_truth, axis=1)
            
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    def backward(self, y_truth, y_probability):
        samples = len(y_probability)
        if len(y_truth.shape) == 1:
            y_truth = np.eye(y_probability.shape[1])[y_truth]
        
        self.dinputs = -y_truth/ y_probability
        self.dinputs = self.dinputs / samples

class Fast_CCE_Loss: #composition
    def __init__(self):
        self.activation = Softmax()
        self.loss = CategoricalCrossEntropyLoss()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(y_true, self.output)

    def backward(self, dvalues, y_truth):
        samples = len(dvalues)
        #one-hot encoding conversion
        if len(y_truth.shape) == 2:
            y_truth = np.argmax(y_truth, axis=1)
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_truth] -= 1
        self.dinputs = self.dinputs / samples

class Gradient_Decent:
    def  __init__(self, learning_rate = 0.5):
        self.learning_rate= learning_rate
        
    def update_parameters(self, layer: DenseLayer):
        layer.weights += -self.learning_rate * layer.dweights
        layer.biases += -self.learning_rate * layer.dbiases
        
# TODO: find a way to better format the layers and activations. maybe do it in a way that
# accepts a dictionary of objects, check for a design pattern

# making a small network
class MyFirstNeuralNetwork:
    def __init__(self, training, y):
        self.data = training
        self.truth = y
        self.input_layer = DenseLayer(2,64)
        self.hidden_layer = DenseLayer(64,3)
        self.activation1 = ReLu()
        self.activation2 = Fast_CCE_Loss()


    def forward(self):
        #layer pass
        self.input_layer.forward(self.data)
        self.activation1.forward(self.input_layer.output)
        
        self.hidden_layer.forward(self.activation1.output)
        self.loss = self.activation2.forward(self.hidden_layer.output, self.truth)
        
        #forward pass results
        self.probabilities = self.activation2.output
        self.predictions = np.argmax(self.probabilities, axis=1)
        if len(self.truth.shape) == 2:
            self.truth = np.argmax(self.truth, axis=1)
        self.accuracy = np.mean(self.predictions == self.truth)
        
    def backward(self):
        self.activation2.backward(self.probabilities, self.truth)
        self.hidden_layer.backward(self.activation2.dinputs)
        self.activation1.backward(self.hidden_layer.dinputs)
        self.input_layer.backward(self.activation1.dinputs)
        
        


In [ ]:
import numpy as np
from nnfs.datasets import spiral_data
import matplotlib.pyplot as plt
plt.subplots()

np.random.seed(42)

X, y = spiral_data(samples=100, classes=3)

plt.scatter(X[:, 0], X[:, 1], c=y, s=100, cmap=plt.cm.Spectral)
plt.xlabel("DATA SET")
plt.show()

nn = MyFirstNeuralNetwork(X, y)
optimizer = Gradient_Decent()

for epoch in range(10000):

    nn.forward()
    print(f'Epoch:{epoch}, Loss:{nn.loss}, Accuracy:{nn.accuracy}')
    nn.backward()
    optimizer.update_parameters(nn.hidden_layer)
    optimizer.update_parameters(nn.input_layer)
    





plt.scatter(X[:, 0], X[:, 1], c= nn.predictions, s=100, cmap=plt.cm.Spectral)
plt.xlabel("PREDICTIONS")
plt.show()



In [ ]:
plt.subplot()
plt.plot(nn.activation2.output)
plt.xlabel("activation layer")
plt.show()
plt.scatter(X[:, 0], X[:, 1], c= nn.predictions, s=100, cmap=plt.cm.Spectral)
plt.xlabel("PREDICTIONS")


In [ ]:
loss = CategoricalCrossEntropyLoss()
tensor1 =np.array([[0.1,0.8,0.1],[0.1,0.5,0.4],[0.8,0.1,0.1]])
truth = np.array([1,2,1])
print(loss.calculate(tensor1, truth)) 


In [ ]:
print(y)

In [ ]:
print(X)

In [ ]:
# Church Success "Churn" - probability of a church meeting curtain criteria given n features.

# total_members, 
# anual_income, member_frequency(mean%vistors/total_members), social_programs, member_engament, social_media_efficiency_score(mean%retention/platforms), is_membership_increasing, members_youth, members_adults, members_senior, members_family


### Gradient Decent

* consider a chain of nerons
    - how can you find the rate each neron changes given an expected output

* Chain rule: by taking the derivative of composite functions - ie f'(g'(x)) we can understand how each funciton influences the next.

* 

## example of composition in python

In [ ]:
# class Component:

#    # composite class constructor
#     def __init__(self):
#         print('Component class object created...')

#     # composite class instance method
#     def m1(self):
#         print('Component class m1() method executed...')


class Composite:

    # composite class constructor
    def __init__(self):

        # creating object of component class
        self.obj1 = Component()
        
        print('Composite class object also created...')

     # composite class instance method
    def m2(self):
      
        print('Composite class m2() method executed...')
    
        # calling m1() method of component class
        self.obj1.m1()


# creating object of composite class
obj2 = Composite()

# calling m2() method of composite class
obj2.m2()

In [ ]:
# import numpy as np
# import nnfs
# import math


# E = math.e

# # sets default configuration for numpy to avoid data type errors
# nnfs.init() 

# np.random.seed(42)

# class DenseLayer:
#     def __init__(self, n_inputs, n_neurons):
#         self.weights = self.weights = np.random.randn(n_inputs, n_neurons) * np.sqrt(2 / n_inputs)
#         self.biases = np.zeros((1, n_neurons))

#     def forward(self, inputs):
#         self.inputs = inputs
#         self.output = np.dot(inputs, self.weights) + self.biases
#     # takes the derivaties of the previous layer
    
#     def backward(self, dvalues):
#         self.dweights = np.dot(self.inputs.T, dvalues)
#         self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
#         self.dinputs = np.dot(dvalues, self.weights.T)
#         print(self.dweights)
#         # print(self.dbiases)
#     def __repr__(self):
#         # construct and return a string that represents the network
#         # architecture
#         return "NeuralNetwork: {}".format("-".join(str(l) for l in self.weights))

# class ReLu:
#     def forward(self, inputs):
#         self.inputs = inputs
#         self.output = np.maximum(0, inputs)
    
#     def backward(self, dvalues):
#         self.dinputs = dvalues.copy()
        
#         #zero the gradient where inputs are negative
#         self.dinputs[self.inputs <= 0] = 0

# class Sigmoid:
#     def forward(self, inputs):
#         self.output = 1/(1 + np.exp(-inputs))

# class Softmax:
#     #safe softmax prevents memory overflow
#     def forward(self, inputs):
#         exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
#         probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
#         self.output = probabilities

# class Loss:
#     def calculate(self, y, output):
#         sample_losses = self.forward(y, output)
#         data_loss = np.mean(sample_losses)
#         return data_loss

# class CategoricalCrossEntropyLoss(Loss):
#     def forward(self, y_truth,  y_probs):
#         samples = len(y_probs)
#         y_pred_clipped = np.clip(y_probs, 1e-7, 1-1e-7)
        
#         if len(y_truth.shape) == 1: #probabilites for target values, only if categorical labels
#             correct_confidences = y_pred_clipped[range(samples), y_truth]
            
#         if len(y_truth.shape) == 2: # Masking for one hot encoding
#             correct_confidences = np.sum(y_pred_clipped*y_truth, axis=1)
            
#         negative_log_likelihoods = -np.log(correct_confidences)
#         return negative_log_likelihoods
    
#     def backward(self, y_truth, y_probability):
#         samples = len(y_probability)
#         if len(y_truth.shape) == 1:
#             y_truth = np.eye(y_probability.shape[1])[y_truth]
        
#         self.dinputs = -y_truth/ y_probability
#         self.dinputs = self.dinputs / samples

# class Fast_CCE_Loss: #composition
#     def __init__(self):
#         self.activation = Softmax()
#         self.loss = CategoricalCrossEntropyLoss()

#     def forward(self, inputs, y_true):
#         self.activation.forward(inputs)
#         self.output = self.activation.output
#         return self.loss.calculate(y_true, self.output)

#     def backward(self, dvalues, y_truth):
#         samples = len(dvalues)
#         #one-hot encoding conversion
#         if len(y_truth.shape) == 2:
#             y_truth = np.argmax(y_truth, axis=1)
#         self.dinputs = dvalues.copy()
#         self.dinputs[range(samples), y_truth] -= 1
#         self.dinputs = self.dinputs / samples

# class Gradient_Decent:
#     def  __init__(self, learning_rate = 0.01):
#         self.learning_rate= learning_rate
        
#     def update_parameters(self, layer: DenseLayer):
#         layer.weights += -self.learning_rate * layer.dweights
#         layer.biases += -self.learning_rate * layer.dbiases
        
# # TODO: find a way to better format the layers and activations. maybe do it in a way that
# # accepts a dictionary of objects, check for a design pattern

# # making a small network
# class MyFirstNeuralNetwork:
#     def __init__(self, training, y):
#         self.data = training
#         self.truth = y # Keep original y as self.truth
#         self.input_layer = DenseLayer(2,3)
#         self.hidden_layer = DenseLayer(3,3)
#         self.activation1 = ReLu()
#         self.activation2 = Fast_CCE_Loss()

#     def forward(self):
#         self.input_layer.forward(self.data)
#         self.activation1.forward(self.input_layer.output)

#         self.hidden_layer.forward(self.activation1.output)
#         # Pass self.truth as is; Fast_CCE_Loss.forward handles 1D/2D conversion for loss calculation
#         self.loss = self.activation2.forward(self.hidden_layer.output, self.truth)

#         self.probabilities = self.activation2.output
#         self.predictions = np.argmax(self.probabilities, axis=1)

#         # Store 1D true labels for accuracy and backward pass explicitly
#         if len(self.truth.shape) == 2:
#             self.truth_labels = np.argmax(self.truth, axis=1)
#         else:
#             self.truth_labels = self.truth # If original y was already 1D
#         self.accuracy = np.mean(self.predictions == self.truth_labels)

#     def backward(self):
#         # Always pass 1D true labels to Fast_CCE_Loss.backward for consistency
#         self.activation2.backward(self.probabilities, self.truth_labels)
#         self.hidden_layer.backward(self.activation2.dinputs)
#         self.activation1.backward(self.hidden_layer.dinputs)
#         self.input_layer.backward(self.activation1.dinputs)
        
